In [133]:
import math
import numpy as np

In [134]:
# Assuming data format: {(x, y, z): 'block_type'}
m = {}  # input map
graph = {}
stateful_graph = {}
Fsafe = 3
jump_height = 1 

In [135]:
world_txt_filename = 'test/world-dump.txt'

In [136]:
with open(world_txt_filename, 'r') as file:
    lines = file.readlines()

In [137]:
for line in lines:
    if line.startswith('b'):
        parts = line.split() 
        x, z, y = map(int, parts[1:4]) 
        block_type = parts[4] 
        m[(x, y, z)] = block_type


In [138]:
def add_edge(x1, y1, z1, x2, y2, z2):
    distance = math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2 + (z1 - z2) ** 2)
    graph.setdefault((x1, y1, z1), []).append(((x2, y2, z2), distance))
    
def is_reachable(x, y, z):
    return m.get((x, y, z)) == 'solid' and m.get((x, y, z + 1)) != 'solid'

In [139]:
def construct_graph():
    for (x, y, z), block_type in m.items():
        # if not is_reachable(x, y, z):
        #     continue
        graph.setdefault((x, y, z), [])
        for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]: # we only consider moving 4 dirs
            for dz in range(jump_height, -Fsafe - 1, -1):
                nx, ny, nz = x + dx, y + dy, z + dz
                if nz < z and m.get((nx, ny, z)) == 'solid':
                    continue
                elif is_reachable(nx, ny, nz):
                    add_edge(x, y, z, nx, ny, nz)
                    break

In [140]:
graph = {}
construct_graph()

In [141]:
graph

{(0, 0, 0): [((1, 0, 0), 1.0), ((0, 1, 0), 1.0)],
 (0, 1, 0): [((0, 2, 0), 1.0), ((0, 0, 0), 1.0)],
 (0, 2, 0): [((1, 2, 0), 1.0), ((0, 1, 0), 1.0)],
 (1, 0, 0): [((2, 0, 0), 1.0), ((0, 0, 0), 1.0)],
 (1, 1, 0): [((2, 1, 0), 1.0),
  ((0, 1, 0), 1.0),
  ((1, 2, 0), 1.0),
  ((1, 0, 0), 1.0)],
 (1, 2, 0): [((2, 2, 0), 1.0), ((0, 2, 0), 1.0)],
 (2, 0, 0): [((1, 0, 0), 1.0), ((2, 1, 0), 1.0)],
 (2, 1, 0): [((2, 2, 0), 1.0), ((2, 0, 0), 1.0)],
 (2, 2, 0): [((1, 2, 0), 1.0), ((2, 1, 0), 1.0)],
 (1, 1, 1): [((2, 1, 0), 1.4142135623730951),
  ((0, 1, 0), 1.4142135623730951),
  ((1, 2, 0), 1.4142135623730951),
  ((1, 0, 0), 1.4142135623730951)],
 (1, 1, 2): [((2, 1, 0), 2.23606797749979),
  ((0, 1, 0), 2.23606797749979),
  ((1, 2, 0), 2.23606797749979),
  ((1, 0, 0), 2.23606797749979)]}

In [150]:
def add_edge_with_resource(x1, y1, z1, k1, x2, y2, z2, k2, ex_cost):
    distance = math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2 + (z1 - z2) ** 2)
    stateful_graph.setdefault(((x1, y1, z1), k1), []).append((((x2, y2, z2), k2), distance + ex_cost))
    
def construct_graph_with_resources(K):
    for (x, y, z), block_type in m.items():
        for k in range(K+1):  
            for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
                for dz in [1]: # can only destruct block at z+1
                    nx, ny, nz = x + dx, y + dy, z + dz
                    # no block or not solid or have resource
                    if m.get((nx, ny, nz)) == 'solid' and k > 0:
                        nk = k - 1
                        m[(nx, ny, nz)] = 'air'
                        if is_reachable(nx, ny, nz-1):
                            add_edge_with_resource(x, y, z, k, nx, ny, nz-1, nk, 1) 
                            m[(nx, ny, nz)] = 'solid'
                for dz in range(jump_height, -Fsafe - 1, -1):
                    nx, ny, nz = x + dx, y + dy, z + dz
                    if is_reachable(nx, ny, nz):
                        add_edge_with_resource(x, y, z, k, nx, ny, nz, k, 0) 
                        break            
           

In [151]:
K = 3
stateful_graph = {}
construct_graph_with_resources(K)

In [152]:
stateful_graph

{((0, 0, 0), 0): [(((1, 0, 0), 0), 1.0), (((0, 1, 0), 0), 1.0)],
 ((0, 0, 0), 1): [(((1, 0, 0), 1), 1.0), (((0, 1, 0), 1), 1.0)],
 ((0, 0, 0), 2): [(((1, 0, 0), 2), 1.0), (((0, 1, 0), 2), 1.0)],
 ((0, 0, 0), 3): [(((1, 0, 0), 3), 1.0), (((0, 1, 0), 3), 1.0)],
 ((0, 1, 0), 0): [(((0, 2, 0), 0), 1.0), (((0, 0, 0), 0), 1.0)],
 ((0, 1, 0), 1): [(((1, 1, 0), 0), 2.0),
  (((0, 2, 0), 1), 1.0),
  (((0, 0, 0), 1), 1.0)],
 ((0, 1, 0), 2): [(((1, 1, 0), 1), 2.0),
  (((0, 2, 0), 2), 1.0),
  (((0, 0, 0), 2), 1.0)],
 ((0, 1, 0), 3): [(((1, 1, 0), 2), 2.0),
  (((0, 2, 0), 3), 1.0),
  (((0, 0, 0), 3), 1.0)],
 ((0, 2, 0), 0): [(((1, 2, 0), 0), 1.0), (((0, 1, 0), 0), 1.0)],
 ((0, 2, 0), 1): [(((1, 2, 0), 1), 1.0), (((0, 1, 0), 1), 1.0)],
 ((0, 2, 0), 2): [(((1, 2, 0), 2), 1.0), (((0, 1, 0), 2), 1.0)],
 ((0, 2, 0), 3): [(((1, 2, 0), 3), 1.0), (((0, 1, 0), 3), 1.0)],
 ((1, 0, 0), 0): [(((2, 0, 0), 0), 1.0), (((0, 0, 0), 0), 1.0)],
 ((1, 0, 0), 1): [(((2, 0, 0), 1), 1.0),
  (((0, 0, 0), 1), 1.0),
  (((1, 

In [153]:
# TODO: below are modified based on chatgpt's solution

import math
from queue import PriorityQueue

# Assuming graph is already defined as shown in previous examples

def euclidean_distance(x1, y1, z1, x2, y2, z2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2 + (z2 - z1) ** 2)

def a_star_search(start, goal):
    open_set = PriorityQueue()
    open_set.put((0, start))
    
    came_from = {}
    g_score = {node: float('inf') for node in graph}
    g_score[start] = 0
    
    f_score = {node: float('inf') for node in graph}
    f_score[start] = euclidean_distance(*start, *goal)
    
    while not open_set.empty():
        current = open_set.get()[1]  # Get node with lowest f_score value
        
        if current == goal:
            path = [current]
            total_cost = g_score[current]  # Total cost to reach the goal
            while current in came_from:
                current = came_from[current]
                path.append(current)
            path.reverse()
            return path, total_cost
        
        for neighbor, distance in graph[current]:
            tentative_g_score = g_score[current] + distance
            
            if tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = g_score[neighbor] + euclidean_distance(*neighbor, *goal)
                open_set.put((f_score[neighbor], neighbor))
    
    return False, 0 



In [154]:
# Example usage
start = (0, 0, 0) 
goal = (2, 2, 0) 
path, cost = a_star_search(start, goal)
print("Path:", path)
print("Total cost:", cost)

Path: [(0, 0, 0), (0, 1, 0), (0, 2, 0), (1, 2, 0), (2, 2, 0)]
Total cost: 4.0


In [155]:
def a_star_search_with_resources(start, goal, K):
    open_set = PriorityQueue()
    # Adjust the start node format to include the resource count within the tuple
    open_set.put((0, (start, K)))  # Start node now properly includes initial resources
    
    came_from = {}
    g_score = {((x, y, z), k): float('inf') for ((x, y, z), k) in stateful_graph}
    g_score[(start, K)] = 0  # Initialize g_score for the start node with resources
    
    f_score = {((x, y, z), k): float('inf') for ((x, y, z), k) in stateful_graph}
    f_score[(start, K)] = euclidean_distance(*start, *goal)  # Initialize f_score
    while not open_set.empty():
        # Adjusted to retrieve ((x, y, z), k) format directly
        current_cost, (current_pos, current_k) = open_set.get()
        
        if current_pos == goal:
            path = [(current_pos, current_k)]
            total_cost = g_score[(current_pos, current_k)]
            while (current_pos, current_k) in came_from:
                current_pos, current_k = came_from[(current_pos, current_k)]
                path.append((current_pos, current_k))
            path.reverse()
            return path, total_cost  # Path reconstructed from goal to start
        
        for (neighbor, k_rest), distance in stateful_graph[(current_pos, current_k)]:
            tentative_g_score = g_score[(current_pos, current_k)] + distance
            
            if tentative_g_score < g_score[(neighbor, k_rest)]:
                # Update the path and scores for better path
                came_from[(neighbor, k_rest)] = (current_pos, current_k)
                g_score[(neighbor, k_rest)] = tentative_g_score
                f_score[(neighbor, k_rest)] = tentative_g_score + euclidean_distance(*neighbor, *goal)
                open_set.put((f_score[(neighbor, k_rest)], (neighbor, k_rest)))
    
    return False 


In [157]:
# Example usage
start = (0, 1, 0) 
goal = (2, 1, 0) 
K = 3
path, cost = a_star_search_with_resources(start, goal, K)
print("Path:", path)
print("Total cost:", cost)

path, cost = a_star_search(start, goal)
print("Path:", path)
print("Total cost:", cost)

Path: [((0, 0, 0), 3), ((0, 1, 0), 3), ((0, 2, 0), 3), ((1, 2, 0), 3), ((2, 2, 0), 3)]
Total cost: 4.0
Path: [(0, 0, 0), (0, 1, 0), (0, 2, 0), (1, 2, 0), (2, 2, 0)]
Total cost: 4.0
